In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# cd drive/My \Drive/.....

In [ ]:
# Testing
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn
import numpy as np
from tqdm.notebook import tqdm

# Define ResidualBlock class for EDSR
class ResidualBlock(nn.Module):
    def __init__(self, num_features):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=3 // 2)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(num_features, num_features, kernel_size=3, padding=3 // 2)

    def forward(self, x):
        residual = x
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        x += residual
        return x

# Define EDSR (Enhanced Deep Super-Resolution) class
class EDSR(nn.Module):
    def __init__(self, scale_factor=2, num_channels=1, num_features=64, num_blocks=16):
        super(EDSR, self).__init__()
        
        self.input = nn.Conv2d(num_channels, num_features, kernel_size=3, padding=3 // 2)
        
        self.residual_blocks = nn.Sequential(*[ResidualBlock(num_features) for _ in range(num_blocks)])

        self.output_conv = nn.Conv2d(num_features, num_features, kernel_size=3, padding=3 // 2)
        
        self.upscale = nn.Sequential(
            nn.Conv2d(num_features, num_features * (scale_factor ** 2), kernel_size=3, padding=3 // 2),
            nn.PixelShuffle(scale_factor),
        )
        
        self.final = nn.Conv2d(num_features, num_channels, kernel_size=3, padding=3 // 2)

    def forward(self, x):
        initial = self.input(x)
        
        residual = self.residual_blocks(initial)
        residual += initial
        
        upscaled = self.upscale(residual)
        output = self.final(upscaled)
        
        return output

device = torch.device("cuda")
model = torch.load('./Weights/EDSR.pth').to(device)

x_test = np.load('./Data/test_LR.npy').astype(np.float32).reshape(-1,1,75,75)

out = []
for i in tqdm(range(x_test.shape[0])):

    data = torch.from_numpy(x_test[i].reshape(1,1,75,75))
    data = data.to(device)
    recon = model(data)
    out.append(recon.cpu().detach().numpy().reshape(1,150,150))
dataSR = np.asarray(out)
print(dataSR.shape)

x = np.load('./Data/test_HR.npy').astype(np.float32).reshape(-1,1,150,150)
x_out = dataSR.astype(np.float32)

print("Metrics:")
criteria = nn.MSELoss()
losses = []
Ssim = []
Psnr = []
for i in range(x_test.shape[0]):
    losses.append(criteria(torch.from_numpy(x_out[i]), torch.from_numpy(x[i])))
    Ssim.append(ssim(x[i][0], x_out[i][0], data_range=x_out[i][0].max() - x_out[i][0].min()))
    Psnr.append(psnr(x[i][0], x_out[i][0]))
print("Average MSE super resolution samples: " + str('%.5f'%np.average(losses)))
print("Average SSIM super resolution samples: " + str('%.5f'%np.average(Ssim)))
print("Average PSNR super resolution samples: " + str('%.5f'%np.average(Psnr)))


x_out2 = np.load('./Data/test_LR_Inter.npy').astype(np.float32).reshape(-1,1,150,150)
print("Metrics for Interpolated images:")
criteria = nn.MSELoss()
losses = []
Ssim = []
Psnr = []
for i in range(x_test.shape[0]):
    losses.append(criteria(torch.from_numpy(x_out2[i]), torch.from_numpy(x[i])))
    Ssim.append(ssim(x[i][0], x_out2[i][0], data_range=x_out2[i][0].max() - x_out2[i][0].min()))
    Psnr.append(psnr(x[i][0], x_out2[i][0]))
print("Average MSE Interpolated samples: " + str('%.5f'%np.average(losses)))
print("Average SSIM Interpolated samples: " + str('%.5f'%np.average(Ssim)))
print("Average PSNR Interpolated samples: " + str('%.5f'%np.average(Psnr)))

In [ ]:
# Visualize samples
import cv2

out = []
for i in tqdm(range(x_test.shape[0])):
    data = x_test[i].reshape(75,75)
    recon = cv2.resize(data, (150,150), interpolation=cv2.INTER_LINEAR) # Bilinear Interpolation
    out.append(recon.reshape(1,150,150))
dataInter = np.asarray(out)
dataLR = x_test
dataHR = x

for i in range(20):
  f, axarr = plt.subplots(nrows=1,ncols=4,figsize=(16,4))
  plt.sca(axarr[0]); 
  plt.imshow(dataLR[i][0]); plt.title('Low Resolution Image (Input)')
  plt.sca(axarr[1]); 
  plt.imshow(dataInter[i][0]); plt.title('Interpolated Image')
  plt.sca(axarr[2]); 
  plt.imshow(dataSR[i][0]); plt.title('Model Output')
  plt.sca(axarr[3]); 
  plt.imshow(dataHR[i][0]); plt.title('Ground Truth')
  plt.savefig('./Results/.../Sample' + str(i+1) + '.png', format='png', dpi=300)
  plt.close()